# CI Portfolio Project 5 - Filter Maintenance Predictor 2022
## **Feature Engineering Notebook**

## Objectives

**1. Cleaning**

Performed within the [data cleaning notebook](https://github.com/roeszler/filter-maintenance-predictor/blob/main/jupyter_notebooks/02_DataCleaning.ipynb).

**2. Data Transformation**
* Processing the data for the modelling stage.
* Transform data into a format that is useful for the algorithm learn the relationship among the variables.
* Evaluate the use of the following approaches to engineer the variables:
    * ordinal categorical encoding
    * numerical transformation
    * smart correlated selection
    
**3. Feature Extraction**
* Evenly distribute dust type

**4. Feature Selection**

**5. Feature Iteration**



### Inputs

1. Cleaned Test Dataset : `outputs/datasets/collection/dfCleanTrain.csv`

2. Cleaned Train Dataset : `outputs/datasets/collection/dfCleanTrain.csv`

### Outputs

* Generate engineered Train and Test sets, both saved under `outputs/datasets/transformed`

### Conclusions

  * Best approach to engineer variables based on...
  * Transformations that we will consider in our pipeline are...

---

# Change working directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("Current directory set to new location")

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Cleaned Data

In [ ]:
import pandas as pd
df_total = pd.read_csv(f'outputs/datasets/cleaned/dfCleanTotal.csv')

---

# Data Transformation
## Feature Extraction

It is important to define
* Feature Extraction: **creates** new features from functions of the original features.
* Feature Selection: allows us to **choose** a subset of the features for use in model construction.

Semantically we could argue each are a **new feature** or a **subset** of the feature. We have however considered any value intended for use in model construction as a **Selected Feature**: *The process of selecting a subset of relevant features (variables, predictors) for use in model construction*. 

Feature selection techniques are used for several reasons:

* Simplification of models to make them easier to interpret by researchers/users
* Shorter training times
* Avoiding too many input variables (dimensionality)
* Improve the data's compatibility with a learning model class
* Create symmetries in the input data.

The main idea when using a feature selection is that the data contains some features that are either **surplus**, **redundant** or **irrelevant** to the final business goal and can therefore be removed without incurring much loss of predictive power.

## Ordinal Categorical encoding

#### Replacing categories with ordinal numbers 
The dataset as two categorical features, **Data_No** and **Dust**.

These have been been transformed in the **01_DataCollection** notebook to assist us with the manipulation of the data.  They are in the format we desire at this point. We may need to transform these back into their respective categorical values, however this won't be performed until required.

In [ ]:
# data_no_total = df_total['Data_No'].map(str)
# df_total['Data_No'] = data_no_total
# df_total.info()

## Numerical transformations

This process can consider transformers like:
* Smooth with SMA
* Logarithmic in base e
* Logarithmic in base 10
* Reciprocal
* Power
* BoxCox
* Yeo Johnson

The dataset has several numerical features to assess. A [custom function](https://github.com/Code-Institute-Solutions/churnometer/blob/main/jupyter_notebooks/04%20-%20FeatureEngineering.ipynb) found at the [code institute] has been included as a quick way to represent the data for evaluation. Including the full report into this notebook made it a little unwieldy. It has been included as an addendum notebook at [04_FeatEngAddendum.ipynb](https://github.com/roeszler/filter-maintenance-predictor/blob/main/jupyter_notebooks/03A_FeatEngAddendum.ipynb) optionally for your review. 

**In summary**, it reveals:
* BoxCox and Yeo Johnson for most variables improve `Time` and `Differential Pressure` features, however the existence of negative **differential pressure** values present a challenge to log transform this data.
* Due to the [value of log transformation](https://stats.stackexchange.com/questions/107610/what-is-the-reason-the-log-transformation-is-used-with-right-skewed-distribution) to regression models, the following notebook follows a modification and filtering of the data to remove negative values and apply a log transformation. 
* Notwithstanding, a **Modified Log Transformed**, **BoxCox** and **Yeo Johnson** will be considered at each modelling stage to evaluate their effects on model performance. 

## Change in Differential Pressure
Include change in Differential Pressure calculation

**Note**: We replace first instance of `change_DP` with first value of `differential_pressure` in the new bin. 
* This signifies that the fist observation starts from a **zero** DP value.

In [ ]:
df_change_dp = pd.DataFrame()

list_data_nos = list(df_total['Data_No'].unique())
for n in list_data_nos:
    if (df_total.Data_No != df_total.Data_No.shift(1)).any().any():
        df_bin = df_total[df_total['Data_No'] == n]

        change_dp_calc = df_bin['Differential_pressure'].diff().fillna(df_bin['Differential_pressure'])
        df_bin.insert(loc=7, column='change_DP', value=change_dp_calc)

        df_change_dp = pd.concat([df_change_dp, df_bin], ignore_index = True)
df_total = df_change_dp
df_total.loc[446:451]

**Note**: Part of the challenge with this data is dealing with a continuous dataset that is comprised of data 'bins'. Each `Data_No` data bin represents an individual test cycle. 

When engineering **descriptive statistics** into the dataset, that typically used a sample of the previous values to indicate a mean or standard deviation, **we need to treat each bin individually** and not progress the first row of each bin incorrectly with the last rows of the previous bin.

To solve this, we use a sequential loop that proceeds through each bin and inserts a progressive descriptive statistic calculation like change in differential pressure (`change_DP`) and appends it to the previous bin. This eventually creates a version of the dataset that includes the new calculation.

### Outliers in differential pressure observations

In each bin we notice that the size and direction of the `change_DP` measure occasionally produces a zero or negative value. This highlights a fluctuation in the **differential pressures**. These may be considered outliers as the pressure gradient across the filter needs time to stabilize. We have considered three main methods to deal with these observations:
* Log transformation
* Winsorize method
* Dropping the outliers

These will be handled in the [feature engineering](https://github.com/roeszler/filter-maintenance-predictor/blob/main/jupyter_notebooks/03_FeatureEngineering.ipynb) notebook

**Random sample** (Bin 96) to plot and inspect change in Differential Pressure distributions

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(20,5))

bin_96 = df_total[df_total['Data_No'] == 96]
sns.boxplot(x = bin_96['change_DP'], ax=ax1)
sns.histplot(x = bin_96['change_DP'], ax=ax2)
sns.lineplot(x=bin_96['Time'], y=bin_96['Differential_pressure'], ax=ax3)
plt.show()

## Smoothing of **Differential Pressure**
*Dealing with outliers in differential pressure observations*

In each bin note `differential_pressure` and `change_DP` observations. Occasionally the measures fluctuate outside of the general trend of the data (outliers). This can be seen below in the **second to last** observation of Data_No bin 98 (index no. 75826) where the recorded value is outside the general trend of the data. 

In [ ]:
df_total[df_total['Data_No'] == 98].tail()

<!-- To quantify such measures, have considered a tolerance of ±200% change in value, however this can be altered depending on what we see when fitting the models. -->

To **smooth** the variability of this measure we can apply a **mean** (or average) value to the data in various ways. 
This attempts to soften the severity of changes seen and reduce the instances of values that are higher / lower than the general trend. It will effectively reduce variability in the **differential pressure** measure, making it easier to model.

We have considered the following methods to deal with outliers:
* Smooth with Simple Mean Average (SMA)
* Exponentially Weighted Mean (EWM)
* Log transformation
* Dropping the outliers
* Winsorize method

For each calculation that uses previous observations to produce a value, we can **use the same process we applied to manage the unique data bins** as we did in calculating change in differential pressure above.

#### Smoothing of Differential Pressure with a **Simple Moving Average** (SMA)
A simple moving average (SMA) is an arithmetic **moving average** calculated by adding recent values (the last four measures in this case), then dividing that by the number of observations in the calculation. This moves along as the observations progresses.

* We can see the first four observations are **NaN** indicated. 
    * These could be imputed with arbitrary values, mean values, closest k sample values and/or a MICE (Multiple Imputation by Chained Equations) value that fits a linear regression with the present values.
    * On review, the MICE method would be the preferred method to impute the missing SMA values, **however**: considering the progressive nature of the `differential_pressure` variable, a preferable alternate to SMA would be Exponentially Weighted Mean (EWM).

#### Smoothing of Differential Pressure with an **Exponentially Weighted Mean** (EWM)
Is a measure of the moving average that considers older observations to have given lower weightings in the calculation. The weights fall exponentially as the data point gets older – hence the name exponentially weighted.

* The EWM is calculated with a 4 point EWM (`span=4`) measure, that considers the previous 4 observations to calculate the weighted mean.


In [ ]:
df_means = pd.DataFrame()

list_data_nos = list(df_total['Data_No'].unique())
for n in list_data_nos:
    if (df_total.Data_No != df_total.Data_No.shift(1)).any().any():
        df_bin = df_total[df_total['Data_No'] == n]

        ewm_calc = df_bin['Differential_pressure'].ewm(span=4, adjust=False).mean()
        df_bin.insert(loc=2, column='4point_EWM', value=ewm_calc)

        sma_calc = df_bin['Differential_pressure'].rolling(4).mean()
        df_bin.insert(loc=2, column='4point_SMA', value=sma_calc)

        change_ewm_calc = df_bin['4point_EWM'].diff().fillna(df_bin['4point_EWM'])
        df_bin.insert(loc=10, column='change_EWM', value=change_ewm_calc)

        df_means = pd.concat([df_means, df_bin], ignore_index = True)
df_total = df_means
df_total.loc[446:451]

## **Logarithmic Transformation** of Differential Pressure

As seen at data collection, the original continuous `differential_pressure` data is right or **positively skewed** at 1.81 and does not follow the shape of a normally distributed bell curve.
* Included a plot for both instances of the **moving average smoothed** variable to check that it is representative of the **differential_pressure** source, which it is.


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(20,6))

sns.histplot(x = df_total['Differential_pressure'], ax=ax1)
sns.histplot(x = df_total['4point_SMA'], ax=ax2)
sns.histplot(x = df_total['4point_EWM'], ax=ax3)

ax1.title.set_text('Differential Pressure Histogram\n')
ax2.title.set_text('Simple Moving Average Histogram\n')
ax3.title.set_text('Exponential Weighted Mean Histogram\n')
plt.show()

A **log transformation** of this data will represent the values within a normal distribution, **as much as possible**, allowing a more valid statistical analysis from this data.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(20,6))

df_total_log_dp = df_total['Differential_pressure']
log_dp = np.log(df_total_log_dp)
sns.histplot(x = log_dp, ax=ax1)

df_total_log_sma = df_total['4point_SMA']
log_sma = np.log(df_total_log_sma)
sns.histplot(x = log_sma, ax=ax2)

df_total_log_ewm = df_total['4point_EWM']
log_ewm = np.log(df_total_log_ewm)
sns.histplot(x = log_ewm, ax=ax3)

# plt.title('Log Histogram')
ax1.title.set_text('Log_DP Histogram\n')
ax2.title.set_text('Log_SMA Histogram\n')
ax3.title.set_text('Log_EWM Histogram\n')
plt.show()

**Observations** 
* The shape of the numerical differential pressure data is **improved** by a natural logarithmic transformation.
    <!-- * Untreated values of 85.71 mean, std.deviation at 114.34 and median at 38.34, indicates the positively skewed nature of the original data
    * Transformed the data is much improved with mean 3.18 much closer to the median 3.89 -->

The transformed data is still affected by the negative skew in the data:
* Pressure measures show values at zero or below at the beginning and tail of the data. This is amplified when we transform the mean data sets.
    * The instances of negative numbers can be managed by:
        * Adding a constant to stop the value becoming negative, or 
        * Indicate the negative as a missing number or 
        * Dropping the entire observation
    * Considering these measures tend to fall in zones at the beginning or end of the test where the procedure takes a while to 'equalize' or has passed the point of filter failure, we have decided to treat them as outliers and confidently drop the entire row these observation sits within without impacting the power of the models.

In [ ]:
# df = pd.DataFrame()
# df.insert(loc=0, column='log_EWM', value=(log_ewm*10))
# df.insert(loc=0, column='log_SMA', value=(log_sma*10))
# df.insert(loc=0, column='log_DP', value=(log_dp*10))
# df.insert(loc=0, column='DP', value=df_total_log_dp)
# round(df.describe(), 2)

**Further EWM transformation**

Remove rows with negative numbers in `4pointWEM` column

In [ ]:
# df_total.insert(loc=4, column='log_EWM', value=log_ewm)
# data = df_total.loc[:, df_total.columns == 'log_EWM']
# df_total = df_total[data.select_dtypes(include=[np.number]).ge(-0).all(1)]
# # df_total

In [ ]:
# Remove Negative values
df_total.insert(loc=4, column='log_EWM', value=log_ewm)
data = df_total.loc[:, df_total.columns == 'log_EWM']
df_total = df_total[data.select_dtypes(include=[np.number]).ge(-0).all(1)]

# Visualize the data
old_shape = pd.read_csv(f'outputs/datasets/cleaned/dfCleanTotal.csv')
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,5))

sns.histplot(x = log_dp, ax=ax1)
sns.histplot(x = df_total['log_EWM'], ax=ax2)

plt.title('log_EWM Histogram')
ax1.title.set_text('Transformed Differential Pressure Histogram\n')
ax2.title.set_text('Transformed Differential Pressure Exponentially Weighted Mean Histogram (Values > 0)\n')
plt.show()
print("Old Shape: ", old_shape.shape)
print("New Shape: ", df_total.shape)

This transformed data is far more useable to train the model. 
* Less variability 
* Negative values removed
* Minimal loss of data

### Further Management of Outliers
Dropping all negative numbers to reduce the noise of the data

#### Evaluation of detecting outliers using **Inter Quartile Range** (IQR).

In [ ]:
df_total_IQR = df_total.copy()

# IQR
Q1 = np.percentile(df_total_IQR, 25, method='midpoint')
Q3 = np.percentile(df_total_IQR, 75,method='midpoint')
IQR = Q3 - Q1
print("Old Shape: ", df_total_IQR.shape)
 
# Upper bound
upper = np.where(df_total_IQR >= (Q3+1.5*IQR))
# Lower bound
lower = np.where(df_total_IQR <= (Q1-1.5*IQR))
 
''' Removing the Outliers '''
df_total_IQR.drop(upper[0], inplace = True)
df_total_IQR.drop(lower[0], inplace = True)
 
print("New Shape: ", df_total_IQR.shape)

# df_total_log_IQR = df_total_IQR['Differential_pressure']
# df_total_log_IQR = df_total_IQR['4point_SMA']
# df_total_log_IQR = df_total_IQR['4point_EWM']
df_total_log_IQR = df_total_IQR['log_EWM']
log_dp_IQR = np.log(df_total_log_IQR)
# df_total_IQR['log_DP'] = log_dp
sns.histplot(x = df_total_log_IQR)
plt.title('IQR Histogram of log EWM')
plt.show()

IQR transformation is not particularly effective on this data with negative values removed. We will not apply this method and proceed with other techniques.

#### **The Winsorize Method**
Winsorization is the process of replacing the extreme values of statistical data in order to limit the effect of the outliers on the calculations or the results obtained by using that data. To apply this to a our exponentially logged differential pressure `log_EWM` variable, where outliers are present only at one end of the data:
* The lower 10% values of the data will have their values set equal to the value of the data point at the 10th percentile.

In [ ]:
from scipy.stats.mstats import winsorize

WinsorizedArrayMean = np.mean(df_total['log_EWM'])
winz_EWM = winsorize(df_total['log_EWM'],(0.1,0.1))
df_total.insert(loc=5, column='winz_EWM', value=winz_EWM)
WinsorizedArray = df_total['winz_EWM']
plt.boxplot(WinsorizedArray)
plt.title('Winsorized array')
plt.show()
WinsorizedArrayNewMean = np.mean(WinsorizedArray)
print('Old Mean: ', WinsorizedArrayMean)
print('New Mean: ', WinsorizedArrayNewMean)

In [ ]:
df_total_log_winz = df_total['winz_EWM']
log_winz = np.log(df_total_log_winz)
sns.histplot(x = log_winz)

Winzorization smooths the data too much and does not add much value to training our models so we will not apply this method either. 

In [ ]:
del df_total['winz_EWM']
del df_total['4point_SMA']
# del df_train['winz_Mean']
# del df_train['log_DP']
df_total

## Evenly distribute dataset by `Dust` type

Both the **train** and **test** sets supplied have data distributed unevenly between 50 test bins. To account for this we wish to assess the measures of central tendency for each Dust class, with the aim of reducing the data size to a more evenly proportioned one between classes.

#### **Train** Dataset

**Considerations**

* The proportion of data that **has reached filter failure** is represented by how close `filter_balance` is to zero or less. 
    * Data with filter_balance values approaching zero may be worth keeping and will make part of our heuristic decision process.
* Notwithstanding that the **mean** is the most frequently used measure of central tendency because it uses all values in the data set to give you an average
    * For data from skewed distributions (like `differential_pressure`), the **median** is better than the mean because it isn’t influenced by extremely large values.

In the following calculation, we see a summary of the top ten `Data_No` bins where `differential_pressure` observations that have made it to the **600 Pa** (the point of filter failure).

Divide back into **Train** and **Test** sets

In [ ]:
data_no_total = df_total['Data_No'].map(int).round(decimals=0)
df_total.loc[:, 'Data_No'] = data_no_total

n = df_total['Data_No'].iloc[0:len(df_total)]
df_train = df_total[n < 51].reset_index(drop=True, names='index')
df_test = df_total[n > 50].reset_index(drop=True, names='index')
del df_train['RUL']
df_test

In [ ]:
last_row_train = df_train[df_train.Data_No != df_train.Data_No.shift(-1)]
# last_row_descending = last_row_train.sort_values(by='Dust', ascending=True)
last_row_descending = last_row_train.sort_values(by='Differential_pressure', ascending=False)
last_row_descending.head(n=10)

Note the diagram below showing proportions of `Dust` variable in the **df_train** dataset.
* It shows a disproportionate mix between classes. This will be the first dataset we tidy up.

In [ ]:
%matplotlib inline

category_totals = df_train.groupby('Dust')['Differential_pressure'].count().sort_values()
category_totals.plot(kind="barh", title='Proportion of Dust Classes in df_train\n', xlabel='\nObservations', ylabel='Dust Class')
category_totals

#### Our next aim is to 
* Fill these bins with data that best represents a central tendency.
* Make the size of each bin around ±**6100** observations (similar to the A4 Coarse Dust class bin) 

#### Procedure
* Include a comparison to how far each `differential_pressure` measure **deviates** or how far it is from the **.median()** value of the bin.
* Ordered by `filter_balance` showing sets with data closest to 600 Pa `differential_pressure`.
* Include comparison to median
* Add a cumulative measure of Data_No's to use as a ranking
* Create a dataframe of the A3 Medium Dust : **1.025**

Add a calculation of Standard Deviation to **df_train** test set and hide several colums for easier viewing

In [ ]:
# del df_train['winz_Mean']
# del df_train['log_DP']
std_group = df_train.groupby('Data_No').std()
std_group.index.name = None
std_group.loc[:,'Data_No'] = std_group.index
map_std = df_train['Data_No'].map(std_group.set_index('Data_No')['Differential_pressure'])
df_train.loc[:,'std_DP'] = map_std
# df_test.loc[363:368]
df_train.loc[30082:30087].style.hide(['Time', 'Dust_feed', 'Flow_rate', 'Dust', 'mass_g', 'cumulative_mass_g', 'Tt'], axis="columns")

Add a calculation of Coefficient of Variation (variance) to **df_train** test set

In [ ]:
import numpy as np
cv = lambda data: np.std(data, ddof=1) / np.mean(data, axis=0) * 100 
var_group = df_train.groupby('Data_No').apply(cv)
var_group.index.name = None
var_group.loc[:,'Data_No'] = var_group.index
map_var = df_train['Data_No'].map(var_group.set_index('Data_No')['Differential_pressure'])
df_train.loc[:,'cv_DP'] = map_var
# df_test.loc[363:368]
df_train.loc[444:453]

The coefficient of variation is an indication of how far the standard deviation is away from the mean. As we can see it does not add value to our understanding of the data, primarily due the the skewed nature of the `differential_pressure` continuous variable. 
* This re-enforces the understanding that descriptive statistics using the mean may not be preferred measure of central tendency.

**Remove Coefficient of Variation and Add Median to df_train**
* Median is the preferred measure of central tendency to observe in a skewed dataset such as this as it is not as affected by larger values.

In [ ]:
del df_train['cv_DP']
median_group = df_train.groupby('Data_No').median()
median_group.index.name = None
median_group.loc[:,'Data_No'] = median_group.index
map_median = df_train['Data_No'].map(median_group.set_index('Data_No')['Differential_pressure'])
df_train.loc[:,'median_DP'] = map_median
# df_train.loc[444:453]
df_train.loc[444:453].head().style.hide(['Time', 'Dust_feed', 'Flow_rate', 'Dust', 'mass_g', 'cumulative_mass_g', 'Tt'], axis="columns")

#### Now we can evaluate the dataframe with just **A3 Dust** in it, ordered by `filter_balance` as a measure of time to filter failure

Map the size of each bin and include a cumulative sum of each **bin size** to help see which data bin that reaches **6100** or more total values.

In [ ]:
bin_sum = df_train.groupby('Data_No')['Data_No'].count().reset_index(name='bin_tot')
map_bin = df_train['Data_No'].map(bin_sum.set_index('Data_No')['bin_tot'])
df_train.loc[:, 'bin_size'] = map_bin

dust_A3 = df_train[df_train['Dust'] == 1.025]
filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3 = filter_A3.sort_values(by='filter_balance', ascending=True)

df_train_A3['c_sum'] = df_train_A3['bin_size'].cumsum()
dn_fb = df_train_A3.loc[:, 'Data_No'].head(14).sort_values(ascending=True).reset_index(drop=True)
df_train_A3.head(14).style.hide(['Time', 'Dust_feed', 'Flow_rate', 'Dust', 'mass_g', 'cumulative_mass_g', 'Tt'], axis="columns")

We can see that in the current dataframe containing only A3 Medium Dust observations, that is ordered by those tests with closest to a completed test to failure:
* **The top 9 data bins (seen at bin 7) would extract a A3 Medium dust training dataset with 7,208 observations**
* We will now perform a further PDA to evaluate the suitability of these further

#### Rank by Standard Deviations, ordered by `std_DP`
The standard deviation is used to measure the spread of values in a sample.

In [ ]:
# dust_A3 = df_train[df_train['Dust'] == 1.025]
# filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3_std = filter_A3.sort_values(by='std_DP', ascending=True)
df_train_A3_std['c_sum'] = df_train_A3_std['bin_size'].cumsum()
dn_sdv = df_train_A3_std.loc[:, 'Data_No'].head(14).sort_values(ascending=True).reset_index(drop=True)
df_train_A3_std.head(14).style.hide(['Time', 'Dust_feed', 'Flow_rate', 'Dust', 'mass_g', 'cumulative_mass_g', 'Tt'], axis="columns")

#### Rank by central tenancy of the Median value, ordered by `median_DP`
* Median = the value of the number in the middle of the dataset

In [ ]:
# dust_A3 = df_train[df_train['Dust'] == 1.025]
# filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3_median = filter_A3.sort_values(by='median_DP', ascending=False)
df_train_A3_median['c_sum'] = df_train_A3_median['bin_size'].cumsum()
dn_mdp = df_train_A3_median.loc[:, 'Data_No'].head(14).sort_values(ascending=True).reset_index(drop=True)
df_train_A3_median.head(14).style.hide(['Time', 'Dust_feed', 'Flow_rate', 'Dust', 'mass_g', 'cumulative_mass_g', 'Tt'], axis="columns")

Review of the **data bin numbers** across descriptors of central tendency that would capture **6100** or more total values:

In [ ]:
dn_review = pd.DataFrame()
dn_review.insert(loc=0, column='by_std_DP', value=dn_sdv)
dn_review.insert(loc=0, column='by_median_DP', value=dn_mdp)
dn_review.insert(loc=0, column='by_filter_balance', value=dn_fb)
dn_review

#### Considerations

* We can see that when we filter by **filter balance** or **median differential pressure** that most bin numbers are contained in both. In fact only bins 1, 16, and 24 are not contained in both, se we can consider these as the first to re-introduce to the data, perhaps when creating the validation set. 
    * This provides us with confidence that these test are higher value for our purposes and promote inclusion into our A3 Dust group.
* Reviewing those values with the lowest **Standard Deviation** inadvertently includes selects values furthest from fully completing a test (ie differential pressure reaching > 600 Pa). In such, these bins will not factor highly in our considerations to include in the dataset. 

---

## Feature Selection

* A **Selected Feature**: *The process of selecting a subset of relevant features (variables, predictors) for use in model construction*. 

Feature selection techniques are used for several reasons:

* Simplification of models to make them easier to interpret by researchers/users
* Shorter training times
* Avoiding too many input variables (dimensionality)
* Improve the data's compatibility with a learning model class
* Create symmetries in the input data.

The main idea when using a feature selection is that the data contains some features that are either **surplus**, **redundant** or **irrelevant** to the final business goal and can therefore be removed without incurring much loss of predictive power.

### Extracting bins from the **Training** dataset

Make a separate frame indicating the bin numbers we wish to extract

In [ ]:
bin_no = df_train_A3['Data_No'].head(9)
bin_no.to_frame()

Use these references to create a dataframe `df_train_cleaned_A3` that is ready for inclusion in our final dataframe `df_train_clean`.
* Note we disregard the cumulative sum measure as it doesn't add value to further calculations

In [ ]:
df_train_copy = df_train
df_train_cleaned_A3 = df_train_copy[df_train_copy['Data_No'].isin(bin_no)]
df_train_cleaned_A3

#### A Quick Review: 

In [ ]:
print('Shape started with: ', dust_A3.shape)
print('Shape we have now: ', df_train_cleaned_A3.shape)

Add these to a new dataset to compare

In [ ]:
dust_A2 = df_train[df_train['Dust'] == 0.900]
dust_A3 = df_train_cleaned_A3
dust_A4 = df_train[df_train['Dust'] == 1.200]

df_train_compare = pd.concat([dust_A2, dust_A3, dust_A4], ignore_index = True)
df_train_compare

In [ ]:
%matplotlib inline

category_totals = df_train_compare.groupby('Dust')['Differential_pressure'].count().sort_values()
category_totals.plot(kind="barh", title='Proportion of Dust Classes in "df_train_compare"\n', xlabel='\nObservations', ylabel='Dust Class')
category_totals

### Repeat procedure with remaining `Dust` classes and **Test** dataset

Extract, Clear and Replace...

In [ ]:
# bin_sum = df_train.groupby('Data_No')['Data_No'].count().reset_index(name='bin_Tot')
# map_bin = df_train['Data_No'].map(bin_sum.set_index('Data_No')['bin_Tot'])
# df_train.loc[:, 'bin_Size'] = map_bin

# df_train.loc[38817:38827]

dust_A2 = df_train[df_train['Dust'] == 0.900]
filter_A2 = dust_A2[dust_A2.Data_No != dust_A2.Data_No.shift(-1)]
df_train_A2 = filter_A2.sort_values(by='filter_balance', ascending=True)
df_train_A2['c_sum'] = df_train_A2['bin_size'].cumsum()
df_train_A2.head(13).style.hide(['Time', 'Dust_feed', 'Flow_rate', 'Dust', 'mass_g', 'cumulative_mass_g', 'Tt'], axis="columns")

In [ ]:
bin_no = df_train_A2['Data_No'].head(9)
bin_no.to_frame()

In [ ]:
df_train_copy = df_train
df_train_cleaned_A2 = df_train_copy[df_train_copy['Data_No'].isin(bin_no)]
df_train_cleaned_A2

In [ ]:
dust_A2 = df_train_cleaned_A2
dust_A3 = df_train_cleaned_A3
dust_A4 = df_train[df_train['Dust'] == 1.200]

df_train_compare = pd.concat([dust_A2, dust_A3, dust_A4], ignore_index = True)
df_train_compare

In [ ]:
%matplotlib inline

category_totals = df_train_compare.groupby('Dust')['Differential_pressure'].count().sort_values()
category_totals.plot(kind="barh", title='Proportion of Dust Classes in "df_train_compare"\n', xlabel='\nObservations', ylabel='Dust Class')
category_totals

### **How much data do we need**?
* At a bin sample of 9 for both A2 and A3 dust, we currently have a little under 15,000 observations to train the ML Models
* Will review this depending on the performance of the models
* Overfitting with too much data vs underfitting with too little is a balance and the qualitative nature of the heuristics surrounding the predictions made
* Nonlinear Algorithms (like clustering) may need more data

Short answer is, **we have plenty of extra data at this point that may be less suited to training the model**. 

* Notwithstanding, it is still live data and may there may be some value toward increasing the power of our ML Models. 
* We also have the option to alter the selected number of dust class bins and possibly augmenting the smaller A4 dust dataset with the **Synthetic Minority Oversampling Technique (SMOTE)** or SMOTE NC for categorical data should we need.

In [ ]:
# df_train_cleaned = df_train_compare
# df_train_cleaned
df_train_transformed = df_train_compare
df_train_transformed

We could run the **test data** through the above process, however it is intended to *test* the models and make sure they’ll work when new data is introduced. We do not want to taylor this data too much as it may promote overfitting the model.

## Other Feature Selection Considerations

Popular filter metrics for **classification problems** are 
* **Correlation-based feature selection**
* Mutual information
* Class separability
    * Error probability
    * Inter-class distance
    * Probabilistic distance
    * Entropy
* **Consistency-based feature selection**
* **Correlation-based feature selection**

#### Correlation Based Feature Selection


In [ ]:
df_train_transformed.head()

Plot relevant relationships to **train** dataset

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_train_corr = df_train_transformed.drop(['Data_No', 'Flow_rate', 'Dust', 'mass_g', 'Dust_feed', 'Tt', 'bin_size', 'median_DP', 'std_DP', 'change_DP', 'change_EWM'], axis="columns")
sns.pairplot(data=df_train_corr)

Plot relevant relationships to **test** dataset (includes RUL)

In [ ]:
df_test_corr = df_test.drop(['Data_No', 'Flow_rate', 'Dust', 'mass_g', 'Dust_feed', 'Tt', 'change_DP', 'change_EWM'], axis="columns")
sns.pairplot(data=df_test_corr)

With a quick look, we notice the same relationships between `differential_pressure`, `RUL` and `Time`, with our engineered variables also indicating influence.

#### Heatmaps for **df_train** engineered dataset

Import ppscore function (Code Institute [Exploratory Data Analysis Tools](https://learn.codeinstitute.net/courses/course-v1:CodeInstitute+DDA101+2021_T4/courseware/468437859a944f7d81a34234957d825b/c8ea2343476c48739676b7f03ba9b08e/) 2022).

In [ ]:
import numpy as np
import ppscore as pps

def heatmap_corr(df, threshold, figsize=(10, 8), font_annot=8):
    """
    Heatmap for pearson (linear) and spearman (monotonic) correlations to 
    visualize only those correlation levels greater than a given threshold.
    """
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={'size': font_annot}, ax=axes,
                    linewidth=0.01, linecolor='WhiteSmoke'
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(10, 8), font_annot=8):
    """
    Heatmap for power predictive score
    PPS == 0 means that there is no predictive power
    PPS < 0.2 often means that there is some relevant predictive power but it is weak
    PPS > 0.2 often means that there is strong predictive power
    PPS > 0.8 often means that there is a deterministic relationship in the data,
    """
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={'size': font_annot},
                         linewidth=0.01, linecolor='WhiteSmoke')
        plt.ylim(len(df.columns), 0)
        plt.show()


def calculate_corr_and_pps(df):
    """
    Calculate the correlations and ppscore of a given dataframe
    """
    df_corr_spearman = df.corr(method='spearman')
    df_corr_pearson = df.corr(method='pearson')

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query('ppscore < 1').filter(['ppscore']).describe().T
    print('PPS threshold - check PPS score IQR to decide threshold for heatmap \n')
    print(pps_score_stats.round(4))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def display_corr_and_pps(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(10, 8), font_annot=8):
    """
    Render the correlations and ppscore heatmaps for a given dataframe
    """
    # print('\n')
    print('To analyze: \n** Colinearity: how the target variable is correlated with the other features (variables)')
    print('** Multi-colinearity: how each feature correlates among themselves (multi-colinearity)')

    print('\n')
    print('*** Heatmap: Pearson Correlation ***')
    print(f'It evaluates the linear relationship between two continuous variables \n'
          f'* A +ve correlation indicates that as one variable increases the other variable tends to increase.\n'
          f'A correlation near zero indicates that as one variable increases, there is no tendency in the other variable to either increase or decrease.\n'
          f'A -ve correlation indicates that as one variable increases the other variable tends to decrease.')
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print('\n')
    print(f'*** Heatmap: Spearman Correlation ***')
    print(f'It evaluates monotonic relationship \n'
          f'Spearman correlation coefficients range from -1 to +1.\n'
          f'The sign of the coefficient indicates whether it is a positive or negative monotonic relationship.\n'
          f'* A positive correlation means that as one variable increases, the other variable also tends to increase.')
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print('\n')
    print('*** Heatmap: Power Predictive Score (PPS) ***')
    print(f'PPS detects linear or non-linear relationships between two columns.\n'
          f'The variable on the x-axis is used to predict the corresponding variable on the y-axis.\n'
          f'The score ranges from 0 (no predictive power) to 1 (perfect predictive power)\n\n'
          f'* PPS == 0 means that there is no predictive power\n'
          f'* PPS < 0.2 often means that there is some relevant predictive power but it is weak\n'
          f'* PPS > 0.2 often means that there is strong predictive power\n'
          f'* PPS > 0.8 often means that there is a deterministic relationship in the data\n')
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

In [ ]:
df_drop = df_train_transformed.drop(['Data_No', 'Flow_rate', 'Dust', 'mass_g', 'Dust_feed', 'Tt', 'bin_size', 'median_DP', 'std_DP', 'change_DP', 'change_EWM'], axis="columns")
df_corr_pearson, df_corr_spearman, pps_matrix = calculate_corr_and_pps(df_drop)

In [ ]:
display_corr_and_pps(df_corr_pearson = df_corr_pearson, df_corr_spearman = df_corr_spearman,
                    pps_matrix = pps_matrix, CorrThreshold = 0, PPS_Threshold =0,
                    figsize=(12,10), font_annot=10
                    )

### Observations
Maintaining the observations we have previously made at 02_DataCleaning notebook, we also note:

* As we would think, there is a strong linear relationship between **Exponential Weighted Mean** (4pointEWM) and **Differential Pressure** (DP), indicating that we can confidently consider 4pointEWM (and its log value) as a proxy for DP.
    * Subsequently there is a strong **inverse** linear relationship between Exponential Weighted Mean (4pointEWM) and **Remaining Filter Balance**.
    
* Cumulative mass and time have a mildly positive relationship to Differential Pressure values, however their power to predict it is considered **none** to very weak at best.

## **SmartCorrelatedSelection** of Variables

[Feature Engine](https://pypi.org/project/feature-engine/) is a python library with multiple transformers to engineer and select features for use in machine learning models. 

`pip install feature-engine`

Applied here to confirm our conclusions, it looks for groups of features that correlate amongst themselves and removes any surplus correlated features since they add the same information to the model.

The transformer finds the groups and drops the features based on the **method**, **threshold** and **selection** method we choose.

For every group of correlated features, the transformer will remove all but one feature.

* Step 1: Create a separate DataFrame, with the variable(s)

In [ ]:
df_engineering = df_total.copy().drop(['change_DP', 'change_EWM'], axis="columns")
df_engineering.head()

* Step 2: Create engineered variables(s) applying the transformation(s)

In [ ]:
from feature_engine.selection import SmartCorrelatedSelection
corr_sel = SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")

corr_sel.fit_transform(df_engineering)
# corr_sel.correlated_feature_sets_
print('Correlated Variables :\n', corr_sel.correlated_feature_sets_)
print('\nFeatures to Drop :\n', corr_sel.features_to_drop_)

On running the transformer, it found 2 groups of correlated features:
1. `'DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 'TechSupport',`
2. `'mass_g', 'Tt', 'Dust_feed', 'RUL'`

* It decided `Differential_pressure` was the most relevant feature to keep in **group 1**.
* It decided `Dust_feed` was the most relevant feature to keep in **group 2**.

### Observations
* Confirmed **Differential Pressure** as a primary feature in both **train** & **test** datasets
* Confirmed **Dust_feed** as a primary feature in the **test** dataset.
* We can include these steps to each ML Pipeline

In [ ]:
df_total

In [ ]:
df_test

In [ ]:
df_train_transformed

In [ ]:
df_train

---

## Save Datasets 
Save the files to /transformed folder

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/transformed')
except Exception as e:
  print(e)

df_train.to_csv(f'outputs/datasets/transformed/dfPreTransformTrain.csv',index=False)
df_train_transformed.to_csv(f'outputs/datasets/transformed/dfTransformedTrain.csv',index=False)
df_test.to_csv(f'outputs/datasets/transformed/dfTransformedTest.csv',index=False)
df_total.to_csv(f'outputs/datasets/transformed/dfTransformedTotal.csv',index=False)

---

# Feature Iteration
In this project, feature iteration has been included as part of each modelling process as there is no one model family that works best for every problem. Feature iteration will however follow a process of repeating a set of tasks to achieve a result.

#### **Fitting Parameters** 
Each model has its own parameters that define it which may change depending on the type of model it is; 
* A regression model may be defined by its feature coefficients
* A decision tree may be defined by its branch locations
* A neural network may be defined by the weights connecting its layers

We can determine the appropriate values for the parameters for each model using iterative methods to find the minimum of a function. This is commonly known the gradient descent or loss (or cost) function and will be implemented using the [Scikit-Learn](https://scikit-learn.org/stable/about.html#citing-scikit-learn) library.

#### **Tuning Hyperparameters**
* Each model sits in a category of models (family) with customizable structures
* Each set of structural choices (hyperparameters) must be chosen before fitting the models parameters.

We can build separate models with different structural choices. We use these to investigate the different results and evaluate which is best for what it is we are trying to achieve. This can be done with **Cross-Validation**, an iterative method for evaluating the performance of models built with a given set of hyperparameters.

When we **train** a model, we:
1. Decide hyperparameters for the model family: e.g. Should the model have a penalty to prevent overfitting?
2. Fit the model parameters to the data: e.g. What are the model coefficients that minimize the loss function?
 
#### **Attending to the Problem**
Techniques to attend to those easy to solve issues (low-hanging-fruit) that improve predictive performance. Things like:
* Trying Different Model Families
* Combining multiple models into an ensemble

#### **Improving Our Data**
**Collecting Better Data**:
* Not included as part of this project, as the data has been artificially sourced, however in the workplace we may look at iterating through the data and or data collection process to improve the depth, quality or quantities of data and even collect alternate variables, like filter brand or all tests to the point of filter failure.

**Engineering Better Features**
* By leveraging on the knowledge of subject matter experts, creating new features from the data to improve the model performance.

---

# Conclusions and Next steps

#### Conclusions: 
* Data has been cleaned and engineered with a variety of methods
* Changes in Differential Pressure, its Exponential Weighted Mean (4pointEWM), Time and dust type seem to be the leading parameters to consider for us to create a predictive model. 

#### Next Steps:
* Regression Models
    * With Classification Models as required
* Cluster Model
* Correlation Study

---